# Capturing Knowledge from BugTracker data

### Populating the Database with mock data

In [1]:
# create database and table schema
! pip install mysql-connector

import mysql.connector

# make sure the host matches your GCP instance along with the password created earlier.
host = "35.223.10.112"
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123"
)

cursor = db.cursor()

# drop/create/insert database
with open('./data_files/lab9/data.sql') as f:
    commands = f.read().split(';')
    f.close()
    print('populating database...')
    for command in commands:
        try:
            if command.strip() != '':
                cursor.execute(command)
        except:
            print('[ERROR]:', command)
    db.commit()
    print('data insert complete')

cursor.close()
db.close()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 35.9 MB/s eta 0:00:0000:010:01
  Preparing metadata (setup.py) ... done
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp37-cp37m-linux_x86_64.whl size=247950 sha256=4abd13b50d0e361ab65b09e5fbf2e68cef348bb3dc0de8549bf00c8193ac1307
  Stored in directory: /home/jupyter/.cache/pip/wheels/42/2f/c3/692fc7fc1f0d8c06b9175d94f0fc30f4f92348f5df5af1b8b7
Successfully built mysql-connector
populating database...
data insert complete


Small test query to make sure the data was populated correctly.

In [2]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor()
# cursor.execute('SELECT t.id, t.title, u.email FROM tickets t INNER JOIN users u ON t.assignee=u.id ORDER BY RAND() LIMIT 5')
cursor.execute('SELECT t.id, t.title, u.email FROM tickets t INNER JOIN users u ON t.assignee=u.id ORDER BY RAND() LIMIT 5')
print(*cursor.fetchall(), sep='\n')

cursor.close()
db.close()

(53, "asset's labeled hearsay E's", 'adora.laden@example.org')
(665, 'chain outing yacks crook', 'jacinthe.sheriff@example.org')
(377, 'built elk boldness pi', 'christoffer.twinbourne@example.org')
(61, 'exile ten clone ewe', 'aeriela.tixall@example.org')
(632, 'stable huh yard clubs', 'aeriela.tixall@example.org')


Here's the CREATE query for the `users` and `tickets` tables in our database.

```sql
CREATE TABLE IF NOT EXISTS users (
  id INT AUTO_INCREMENT PRIMARY KEY,
  first_name VARCHAR(30),
  last_name VARCHAR(30),
  email VARCHAR(80) UNIQUE
) ENGINE=InnoDB;

CREATE TABLE IF NOT EXISTS tickets (
  id INT AUTO_INCREMENT PRIMARY KEY,
  title VARCHAR(128),
  project VARCHAR(64) NOT NULL,
  assignee INT,
  description TEXT,
  date_created DATETIME DEFAULT CURRENT_TIMESTAMP,
  date_updated DATETIME DEFAULT CURRENT_TIMESTAMP,
  date_completed DATETIME NULL DEFAULT NULL,
  FOREIGN KEY fk_assignee_id(assignee) REFERENCES users(id)
) ENGINE=InnoDB;
```

To successfully conclude this test, we'll need to answer the following questions:

- How many tickets are open/closed?
- Who has the most tickets assigned?
- What is the longest time taken to complete a task?
- What is the average time to close a ticket?
- How many tickets we're completed in less than 10 days?
- Find tickets that took _more than 80 days_ to complete.
- How can you find tickets that were updated before July 10th, 2020 but not more than one month before?

One requirement is to answer this question doing all of the computations within MySQL, and python will only be our conduit to execute our queries.

## Question 1: _How many tickets are open/closed?_

In [3]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

# get number of tickets open
cursor.execute('SELECT COUNT(*) AS tickets_open FROM tickets t WHERE t.date_completed IS NULL')
tickets_open = cursor.fetchone()['tickets_open']

# get number of tickets closed
cursor.execute('SELECT COUNT(*) AS tickets_closed FROM tickets t WHERE t.date_completed IS NOT NULL')
tickets_closed = cursor.fetchone()['tickets_closed']

print('Tickets Open: {}\nTickets Closed: {}'.format(tickets_open, tickets_closed))

cursor.close()
db.close()

Tickets Open: 413
Tickets Closed: 587


### Using a Single Query

In [4]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

# Enter the single line query SQL below using cursor.execute()
cursor.execute("SELECT SUM(CASE WHEN t.date_completed IS NULL THEN 1 ELSE 0 END) AS 'tickets_open', SUM(CASE WHEN t.date_completed IS NULL THEN 0 ELSE 1 END) AS 'tickets_closed' FROM tickets t")

result = cursor.fetchone()

# get number of tickets open
tickets_open = result['tickets_open']

# get number of tickets closed
tickets_closed = result['tickets_closed']

print('Tickets Open: {}\nTickets Closed: {}'.format(tickets_open, tickets_closed))

cursor.close()
db.close()

Tickets Open: 413
Tickets Closed: 587


## Question 2: _Who has the most tickets assigned?_

In [56]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

query = '''
SELECT
    CONCAT(u.first_name, ' ', u.last_name) AS name,
    u.email, 
    COUNT(t.id) AS ticket_count
FROM tickets t
INNER JOIN users u ON u.id=t.assignee 
GROUP BY t.assignee 
ORDER BY ticket_count DESC
LIMIT 1;
'''

# execute the query
cursor.execute(query)

print(*cursor.fetchall(), sep='\n')
cursor.close()
db.close()

{'name': 'Haskell Cardenoso', 'email': 'haskell.cardenoso@example.org', 'ticket_count': 64}


### Question 2.1: _Update the SQL query to determine who has the _most closed tickets_ and who has the _most open tickets_.

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

query = '''
SELECT
    CONCAT(u.first_name, ' ', u.last_name) AS name,
    u.email, 
    SUM(CASE WHEN t.date_completed IS NOT NULL THEN 1 ELSE 0 END) AS closed_tickets
FROM tickets t
INNER JOIN users u ON u.id=t.assignee 
GROUP BY t.assignee 
ORDER BY closed_tickets DESC
LIMIT 1;
'''

# execute the query
cursor.execute(query)

print(*cursor.fetchall(), sep='\n')
cursor.close()
db.close()

## Question 3: _What is the longest time taken to complete a task?_

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

# query to answer the question
query = '''
SELECT
    DATEDIFF(t.date_completed, t.date_created) AS completed
FROM tickets t
WHERE t.date_completed IS NOT NULL
ORDER BY completed DESC
LIMIT 1
'''

cursor.execute(query)

print(*cursor.fetchall(), sep='\n')
cursor.close()
db.close()

{'completed': 90}


## Question 4: _What is the average time to close a ticket?_

In [61]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

# query to answer the question
query = '''
SELECT
    AVG( DATEDIFF(t.date_completed, t.date_created) ) AS avg_completion_time
FROM tickets t 
WHERE t.date_completed IS NOT NULL
'''

cursor.execute(query)

print(*cursor.fetchall(), sep='\n')
cursor.close()
db.close()

{'avg_completion_time': Decimal('44.9029')}


## Question 4.1: _Notice how the result has four decimal places? Use MySQL to format the result to two decimal places._

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

# query to answer the question
query = '''
SELECT
    TRUNCATE( AVG( DATEDIFF(t.date_completed, t.date_created) ), 2) AS avg_completion_time
FROM tickets t 
WHERE t.date_completed IS NOT NULL
'''

cursor.execute(query)

print(*cursor.fetchall(), sep='\n')
cursor.close()
db.close()

## Question 5: _How many tickets were completed in less than 10 days?_

In [81]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

# query to answer the question
query = '''
SELECT COUNT(*) ticket_count 
FROM (
    SELECT
        t.id,
        DATEDIFF(t.date_completed, t.date_created) AS completed
    FROM tickets t 
    WHERE t.date_completed IS NOT NULL 
    HAVING completed <= 10
) AS quickly_completed_tickets
'''

cursor.execute(query)

print(*cursor.fetchall(), sep='\n')
cursor.close()
db.close() 

{'ticket_count': 73}

### Question 6: _Find tickets that took _more than 80 days_ to complete._

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

# query to answer the question
query = '''
SELECT COUNT(*) ticket_count 
FROM (
    SELECT
        t.id,
        DATEDIFF(t.date_completed, t.date_created) AS completed
    FROM tickets t 
    WHERE t.date_completed IS NOT NULL 
    HAVING completed > 80
) AS quickly_completed_tickets
'''

cursor.execute(query)

print(*cursor.fetchall(), sep='\n')
cursor.close()
db.close()

### Question 7: _How can you find tickets that were updated before July 10th, 2020 but not more than one month before?_

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

# query to answer the question
query = '''
    SELECT *
    FROM tickets
    WHERE date_updated > '2020-06-10' AND date_updated < '2020-07-10'
'''

cursor.execute(query)

print(*cursor.fetchall(), sep='\n')
cursor.close()
db.close()

---